In [12]:
import spacy
import os
import time
import pandas as pd

import multiprocessing

from gensim.models import Word2Vec

In [9]:
!pip install -U spacy

In [10]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 12.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [13]:
# Lista para almacenar las líneas del archivo
lineas = []

# Ruta del archivo de texto
file_path = '/content/drive/MyDrive/PIA/Datasets/WikiQA-train.txt'

# Abrir el archivo en modo lectura
with open(file_path, 'r') as file:
    # Leer cada línea del archivo
    for line in file:
        # Eliminar el carácter de nueva línea al final de cada línea y agregar a la lista
        lineas.append(line.strip())

# Crear DataFrame con la lista de líneas
df = pd.DataFrame({'Text': lineas})

# Imprimir las primeras filas del DataFrame
print(df.head())

                                                Text
0  how are glacier caves formed ?\tA partly subme...
1  how are glacier caves formed ?\tThe ice facade...
2  how are glacier caves formed ?\tIce formations...
3  how are glacier caves formed ?\tA glacier cave...
4  how are glacier caves formed ?\tGlacier caves ...


In [3]:
# Cargando modelo en ingles con sus pesos para el NLP
nlp = spacy.load("en_core_web_md")

In [30]:
def tokenizar_data(data):
  start = time.perf_counter()
  tokens = []
  # Etiquetas que quiero eliminar del texto
  removal= ['PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM ']
  for summary in nlp.pipe(data):
      proj_tok = [token.lemma_.lower() for token in summary if token.pos_ not in removal and not token.is_stop and token.is_alpha]
      tokens.append(proj_tok)
      print(proj_tok)
  print(tokens)
  end = time.perf_counter()
  print(f' Tiempo: {end - start}')
  return tokens

In [17]:
tokens = tokenizar_data(df['Text'])

 Tiempo: 117.53836743700003


In [38]:
model = Word2Vec(min_count=20,
                     window=5,
                     vector_size=300,
                     sample=6e-5,
                     alpha=0.03,
                     min_alpha=0.0007,
                     negative=20,
                     workers=4)#cores-1

In [ ]:
lstSentencesClean = tokenizar_data(df['Text'])

In [37]:
print(lstSentencesClean[0])

['glacier', 'cave', 'form', 'partly', 'submerge', 'glacier', 'cave', 'perito', 'moreno', 'glacier']


In [40]:
start = time.perf_counter()

model.build_vocab(lstSentencesClean, progress_per=10000)

end = time.perf_counter()

print(f' Tiempo para construir vocab: {round((end - start)/60,2)}')

 Tiempo para construir vocab: 0.0


In [41]:
start = time.perf_counter()

model.train(lstSentencesClean, total_examples=model.corpus_count, epochs=30)
end = time.perf_counter()

print(f' Tiempo para entrenar modelo: {round((end - start)/60,2)}')

 Tiempo para entrenar modelo: 0.41


In [42]:
if model.wv.__contains__('florida'):
    print ( model.wv.most_similar('florida', topn=10) ) # obtener palabras similares

[('tampa', 0.789090096950531), ('senator', 0.6845144629478455), ('wisconsin', 0.6794496178627014), ('texas', 0.6340262293815613), ('minnesota', 0.6178409457206726), ('populous', 0.6101747155189514), ('legislature', 0.6093478202819824), ('iowa', 0.5964128375053406), ('beach', 0.5958604216575623), ('miami', 0.5924002528190613)]
